In [1]:
!pip install matplotlib
!pip install koeda
!pip install transformers

In [2]:
import json

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from tqdm import tqdm
from transformers import AutoTokenizer

In [3]:
train = pd.read_csv("train.csv")

NLP에서 Text Augmentation 방법은 크게 텍스트의 일부를 변형하여 데이터를 증강하는 방법과 생성모델을 사용하여 새로운 텍스트를 생성하여 데이터를 증강하는 방법이 있다. 
그 중에서 가장 손쉽게 접근할 수 있는 방법은 KoEDA 라이브러리를 사용하는 것이었다.
KoEDA는 EDA와 AEDA 논문에서 소개된 방식을 한국어 Wordnet 으로 Porting하여 공개한 오픈소스 라이브러리이다.

<EDA의 네 가지 기법> 

Easy Data Augmentation라는 논문에서는 데이터를 다음의 네 가지 기법을 통해 자연어 데이터를 증강하고자 한다.

* 유의어로 교체(Synonym Replacement, SR): 문장에서 랜덤으로 stop words가 아닌 n 개의 단어들을 선택해 임의로 선택한 동의어들 중 하나로 바꾸는 기법.

* 랜덤 삽입(Random Insertion, RI): 문장 내에서 stop word를 제외한 나머지 단어들 중에서, 랜덤으로 선택한 단어의 동의어를 임의로 정한다. 그리고 동의어를 문장 내 임의의 자리에 넣는걸 n번 반복한다.

* 랜덤 교체(Random Swap, RS): 무작위로 문장 내에서 두 단어를 선택하고 위치를 바꾼다. 이것도 n번 반복

* 랜덤 삭제(Random Deletion, RD): 확률 p를 통해 문장 내에 있는 각 단어들을 랜덤하게 삭제한다.

<EDA의 성능>

데이터셋이 적다는 가정에서, 데이터셋이 500개일 때 EDA를 포함하면 평균적으로 3%의 성능이 증가함을 확인할 수 있다. full set일 때도 EDA를 사용하면 평균적으로 성능의 향상이 있었다. 

하지만 우리가 사용할 BERT 등의 선학습 모델은 거대 데이터셋으로 선학습되었기에 데이터셋의 개선 효과를 못 볼 수도 있다고 한다.

<EDA 활용 팁>

한 문장에 대해서 몇 개의 문장을 만들건지에 따라 α값에 조정이 필요하며, 
4문장 이하는 p=0.1, 4문장 초과는 p=0.05 정도의 확률값으로 데이터를 변형하는게 가장 성능이 좋았다고 저술되어있다. 

하지만 텍스트 데이터의 특성상, 위치를 바꾸거나 일부 단어를 제거하는 것은 결국 본 문장의 의미를 손실시키는 행위이기 때문에 AEDA 방법론이 등장하게 된다.

<AEDA란>
AEDA는 문장을 손실시키지 않게 하기 위해 Special character를 문장 곳곳에 배치하는 방법론으로, 
역시 많은 특수문자가 들어가게 되면 성능이 떨어지기 때문에 적절한 확률값을 찾는 것이 중요하다.

***한글 자연어 처리 패키지 konlpy

konlpy는 형태소 등을 알아서 분석해주는 편리한 패키지이지만, konlpy 내 클래스는 Java 기반이기 때문에 그냥 pip install konlpy로 설치할 수 없다. 
아래와 같은 과정을 거쳐야 하는데

1. JAVA 설치
   https://www.oracle.com/java/technologies/javase-downloads.html
2. JAVA_HOME 환경변수 설정
3. JPype 다운로드 및 설치
   https://www.lfd.uci.edu/~gohlke/pythonlibs/#jpype
   파이썬 버전과 맞게 다운로드해야지 cmd창에서 'not a valid wheel filename' 에러가 안 뜬다
4. konlpy 설치

참고: https://byeon-sg.tistory.com/entry/자연어-처리-konlpy-설치-오류-okt에러-already-loaded-in-another-classloader-SystemErro-1 [wave:티스토리]

In [4]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.


In [5]:
import konlpy 
from konlpy.tag import Okt 
okt = Okt()

In [6]:
# EDA 사용방법 (참고: https://github.com/toriving/KoEDA)
from koeda import EDA

eda = EDA(
    morpheme_analyzer="Okt", alpha_sr=0.3, alpha_ri=0.3, alpha_rs=0.2, prob_rd=0.05
)

text = "아버지가 방에 들어가신다"

result = eda(text)
print(result)

아버지가 방에 아빠 들어가신다


In [7]:
# AEDA 사용방법
from koeda import AEDA

aeda = AEDA(
    morpheme_analyzer="Okt", punc_ratio=0.3, punctuations=[".", ",", "!", "?", ";", ":"]
)

text = "어머니가 집을 나가신다"

result = aeda(text)
print(result)

어머니가 ; 집을 나가신다


NLP는 discrete한 성질 때문에 Augmentation에 어려움이 존재한다.
문장의 의미에 큰 영향을 끼치는 단어를 랜덤 삽입/교체/삭제할 수 도 있기 때문에 학습에 효과적이지 않을 것이라고 생각한다.
따라서 AEDA(원 문장의 Sequence length를 기준으로 1/3 ~ 1 사이의 숫자를 랜덤으로 선택하여 6가지 punctuation (".", ";", "?", ":", "!", ",") 중 하나를 랜덤으로 대체시키는 기법)을 사용하여 Augmentation을 진행하였다.


0~5점으로 이루어진 label 데이터를 반올림하여 0/1/2/3/4/5로 바인딩한 라벨을 만들었다.
0~2점까지를 negative pair, 3~5점까지를 positive pair로 보았다.
가장 갯수가 제일 많은 0점 2931개의 문장쌍을 기준으로 나머지 1/2/3/4/5점 문장쌍도 2931개가 되도록 Augmentation을 진행하였다.
바인딩한 라벨 0~5점 각각 2931개로, positive pair과 negative pair의 균형도 맞추고자 하였다. (전체 17586개의 문장쌍) 

그렇게 진행한 결과 raw data로 했을 때보다 Pearson 상관계수가 0.017 증가하였다. (Pearson : 0.9054 → 0.9071)
- AEDA data
- beomi/KcELECTRA-base
- L1Loss
- Batch Size: 32 / Lr: 1e-5 / optimizer: AdamW / Scheduler: StepLR / Epochs: 30

In [8]:
from koeda import AEDA
import pandas as pd
import numpy as np
import copy


def get_preprocessed_label(df):
    """_summary_

    Args:
        df (_type_): _description_

    Returns:
        _type_: _description_
    """
    for i in range(len(df)):
        df.loc[i, "preprocessed_label"] = round(df.loc[i, "label"])

    return df


def concat_AEDA_sent(df_label, aug_nums):
    """라벨 x만 있는 데이터프레임에 aug_nums 만큼 AEDA된 데이터를 추가

    Args:
        df_label (DataFrame): 라벨 x만 있는 데이터프레임
        aug_nums (int): 증강하고자 하는 수
    Returns:
        AEDA가 추가된 라벨 x의 데이터프레임
    """
    np.random.seed(12)
    #aug_idx = np.random.randint(0, 3, size=aug_nums)
    aug_idx = [2]*aug_nums
    dataset_idx = 0
    aug_df_label = copy.deepcopy(df_label)

    aeda = AEDA(
        morpheme_analyzer="Okt",
        punc_ratio=0.3,
        punctuations=[".", ",", "!", "?", ";", ":"],
    )

    for i in range(aug_nums):
        if dataset_idx >= len(df_label):
            dataset_idx = 0

        origin_id = df_label.iloc[dataset_idx]["id"]
        origin_source = df_label.iloc[dataset_idx]["source"]
        origin_sentence_1 = df_label.iloc[dataset_idx]["sentence_1"]
        origin_sentence_2 = df_label.iloc[dataset_idx]["sentence_2"]
        origin_label = df_label.iloc[dataset_idx]["label"]
        origin_binarylabel = df_label.iloc[dataset_idx]["binary-label"]
        origin_preprocessed_label = df_label.iloc[dataset_idx]["preprocessed_label"]

        if aug_idx[i] == 0:  # sent_1만 aug
            sent = aug_df_label.iloc[dataset_idx]["sentence_1"]
            aug_sent = aeda(sent)
            new_df = pd.DataFrame(
                {
                    "id": [origin_id],
                    "source": [origin_source],
                    "sentence_1": [aug_sent],
                    "sentence_2": [origin_sentence_2],
                    "label": [origin_label],
                    "binary-label": [origin_binarylabel],
                    "preprocessed_label": [origin_preprocessed_label],
                }
            )
            aug_df_label = pd.concat([aug_df_label, new_df], ignore_index=True)

        elif aug_idx[i] == 1:  # sent_2만 aug
            sent = aug_df_label.iloc[dataset_idx]["sentence_2"]
            aug_sent = aeda(sent)
            new_df = pd.DataFrame(
                {
                    "id": [origin_id],
                    "source": [origin_source],
                    "sentence_1": [origin_sentence_1],
                    "sentence_2": [aug_sent],
                    "label": [origin_label],
                    "binary-label": [origin_binarylabel],
                    "preprocessed_label": [origin_preprocessed_label],
                }
            )
            aug_df_label = pd.concat([aug_df_label, new_df], ignore_index=True)

        else:  # sent_1과 2를 모두 aug
            sent_1 = aug_df_label.iloc[dataset_idx]["sentence_1"]
            sent_2 = aug_df_label.iloc[dataset_idx]["sentence_2"]
            aug_sent_1 = aeda(sent_1)
            aug_sent_2 = aeda(sent_2)
            new_df = pd.DataFrame(
                {
                    "id": [origin_id],
                    "source": [origin_source],
                    "sentence_1": [aug_sent_1],
                    "sentence_2": [aug_sent_2],
                    "label": [origin_label],
                    "binary-label": [origin_binarylabel],
                    "preprocessed_label": [origin_preprocessed_label],
                }
            )
            aug_df_label = pd.concat([aug_df_label, new_df], ignore_index=True)

        dataset_idx += 1

    return aug_df_label


def AEDA_data():

    train = pd.read_csv("train.csv")
    train = get_preprocessed_label(train)

    train_0 = train[train["preprocessed_label"] == 0].reset_index(drop=True)
    train_1 = train[train["preprocessed_label"] == 1].reset_index(drop=True)
    train_2 = train[train["preprocessed_label"] == 2].reset_index(drop=True)
    train_3 = train[train["preprocessed_label"] == 3].reset_index(drop=True)
    train_4 = train[train["preprocessed_label"] == 4].reset_index(drop=True)
    train_5 = train[train["preprocessed_label"] == 5].reset_index(drop=True)
    
    train_1_aug = concat_AEDA_sent(train_1, 1323)
    train_2_aug = concat_AEDA_sent(train_2, 1906)
    train_3_aug = concat_AEDA_sent(train_3, 1647)
    train_4_aug = concat_AEDA_sent(train_4, 936)
    train_5_aug = concat_AEDA_sent(train_5, 2750)

    train_0 =  pd.concat([train_0, train_1_aug, train_2_aug, train_3_aug, train_4_aug, train_5_aug], ignore_index=True)
    
    auged_df = train_0.sample(frac=1).reset_index(drop=True)
    
    return auged_df

In [9]:
final_df = AEDA_data()
final_df2 = final_df.copy()
final_df

,id,source,sentence_1,sentence_2,label,binary-label,preprocessed_label
0,boostcamp-sts-v1-train-8864,petition-rtt,이니굿즈 많이만 ? 들어주세요,! 이니굿 . 즈 많이 만들어주세요,5.0,1.0,5.0
1,boostcamp-sts-v1-train-864,slack-rtt,담백하고 ! 깔끔한 : 행사 였습니다.,간단하고 ; 깔끔한 행사였습니다.,3.4,1.0,3.0
2,boostcamp-sts-v1-train-5751,slack-rtt,"앞으로 추천팀과 전략팀이 어떻게 더 원활히 협업하고 서로에게 도움을 줄지 이야기 나누며,",추천팀과 전략팀이 앞으로 어떻게 더 잘 협력하고 서로를 도울 수 있는지에 대해 이야...,4.2,1.0,4.0
3,boostcamp-sts-v1-train-4196,slack-sampled,앞 : 으로 ! 잘 부탁드립니다!,앞으로 잘 부탁드립니다 . !!,4.8,1.0,5.0
4,boostcamp-sts-v1-train-4672,petition-sampled,강의보조인력을 늘려주십시오,대학입학에서 정시전형 비율을 늘려주십시오,0.0,0.0,0.0
...,...,...,...,...,...,...,...
17881,boostcamp-sts-v1-train-3583,nsmc-sampled,스토리도 좋았지만 결말이 안타깝네요.,마지막 결말이 아쉬웟지만...,2.8,1.0,3.0
17882,boostcamp-sts-v1-train-4274,nsmc-sampled,영화관에서 아버지가 우는 모습을 처음 봤다.,유일하게 내가 졸면서 보았던 영화.. 그것도 영화관에서.,1.0,0.0,1.0
17883,boostcamp-sts-v1-train-3311,petition-sampled,이국 ; 종 . 교수 ! 님을 적극 지원해 주세요,"이국종 교수 ? 의견 ? 을 , 적극 반영하세요",2.0,0.0,2.0
17884,boostcamp-sts-v1-train-6518,slack-rtt,조만간 먹어봐야겠다는 ; 생각을 했습니다 ; !ㅋㅋㅋ,"빨리 , 해봐야겠다고 생각 . 했어요!",1.0,0.0,1.0


In [10]:
final_df2.to_csv('./train_auged1.csv', index=False)

In [11]:
train = pd.read_csv("./train_auged1.csv")
train

,id,source,sentence_1,sentence_2,label,binary-label,preprocessed_label
0,boostcamp-sts-v1-train-8864,petition-rtt,이니굿즈 많이만 ? 들어주세요,! 이니굿 . 즈 많이 만들어주세요,5.0,1.0,5.0
1,boostcamp-sts-v1-train-864,slack-rtt,담백하고 ! 깔끔한 : 행사 였습니다.,간단하고 ; 깔끔한 행사였습니다.,3.4,1.0,3.0
2,boostcamp-sts-v1-train-5751,slack-rtt,"앞으로 추천팀과 전략팀이 어떻게 더 원활히 협업하고 서로에게 도움을 줄지 이야기 나누며,",추천팀과 전략팀이 앞으로 어떻게 더 잘 협력하고 서로를 도울 수 있는지에 대해 이야...,4.2,1.0,4.0
3,boostcamp-sts-v1-train-4196,slack-sampled,앞 : 으로 ! 잘 부탁드립니다!,앞으로 잘 부탁드립니다 . !!,4.8,1.0,5.0
4,boostcamp-sts-v1-train-4672,petition-sampled,강의보조인력을 늘려주십시오,대학입학에서 정시전형 비율을 늘려주십시오,0.0,0.0,0.0
...,...,...,...,...,...,...,...
17881,boostcamp-sts-v1-train-3583,nsmc-sampled,스토리도 좋았지만 결말이 안타깝네요.,마지막 결말이 아쉬웟지만...,2.8,1.0,3.0
17882,boostcamp-sts-v1-train-4274,nsmc-sampled,영화관에서 아버지가 우는 모습을 처음 봤다.,유일하게 내가 졸면서 보았던 영화.. 그것도 영화관에서.,1.0,0.0,1.0
17883,boostcamp-sts-v1-train-3311,petition-sampled,이국 ; 종 . 교수 ! 님을 적극 지원해 주세요,"이국종 교수 ? 의견 ? 을 , 적극 반영하세요",2.0,0.0,2.0
17884,boostcamp-sts-v1-train-6518,slack-rtt,조만간 먹어봐야겠다는 ; 생각을 했습니다 ; !ㅋㅋㅋ,"빨리 , 해봐야겠다고 생각 . 했어요!",1.0,0.0,1.0


In [12]:
train.columns

Index(['id', 'source', 'sentence_1', 'sentence_2', 'label', 'binary-label',
       'preprocessed_label'],
      dtype='object')

In [13]:
train_copy = train.copy()
train['sentence_1'] = train_copy['sentence_2']
train['sentence_2'] = train_copy['sentence_1']
train

,id,source,sentence_1,sentence_2,label,binary-label,preprocessed_label
0,boostcamp-sts-v1-train-8864,petition-rtt,! 이니굿 . 즈 많이 만들어주세요,이니굿즈 많이만 ? 들어주세요,5.0,1.0,5.0
1,boostcamp-sts-v1-train-864,slack-rtt,간단하고 ; 깔끔한 행사였습니다.,담백하고 ! 깔끔한 : 행사 였습니다.,3.4,1.0,3.0
2,boostcamp-sts-v1-train-5751,slack-rtt,추천팀과 전략팀이 앞으로 어떻게 더 잘 협력하고 서로를 도울 수 있는지에 대해 이야...,"앞으로 추천팀과 전략팀이 어떻게 더 원활히 협업하고 서로에게 도움을 줄지 이야기 나누며,",4.2,1.0,4.0
3,boostcamp-sts-v1-train-4196,slack-sampled,앞으로 잘 부탁드립니다 . !!,앞 : 으로 ! 잘 부탁드립니다!,4.8,1.0,5.0
4,boostcamp-sts-v1-train-4672,petition-sampled,대학입학에서 정시전형 비율을 늘려주십시오,강의보조인력을 늘려주십시오,0.0,0.0,0.0
...,...,...,...,...,...,...,...
17881,boostcamp-sts-v1-train-3583,nsmc-sampled,마지막 결말이 아쉬웟지만...,스토리도 좋았지만 결말이 안타깝네요.,2.8,1.0,3.0
17882,boostcamp-sts-v1-train-4274,nsmc-sampled,유일하게 내가 졸면서 보았던 영화.. 그것도 영화관에서.,영화관에서 아버지가 우는 모습을 처음 봤다.,1.0,0.0,1.0
17883,boostcamp-sts-v1-train-3311,petition-sampled,"이국종 교수 ? 의견 ? 을 , 적극 반영하세요",이국 ; 종 . 교수 ! 님을 적극 지원해 주세요,2.0,0.0,2.0
17884,boostcamp-sts-v1-train-6518,slack-rtt,"빨리 , 해봐야겠다고 생각 . 했어요!",조만간 먹어봐야겠다는 ; 생각을 했습니다 ; !ㅋㅋㅋ,1.0,0.0,1.0


In [14]:
result = pd.concat([final_df, train])
result
result.to_csv('./train_auged2.csv', index=False)

In [15]:
result

,id,source,sentence_1,sentence_2,label,binary-label,preprocessed_label
0,boostcamp-sts-v1-train-8864,petition-rtt,이니굿즈 많이만 ? 들어주세요,! 이니굿 . 즈 많이 만들어주세요,5.0,1.0,5.0
1,boostcamp-sts-v1-train-864,slack-rtt,담백하고 ! 깔끔한 : 행사 였습니다.,간단하고 ; 깔끔한 행사였습니다.,3.4,1.0,3.0
2,boostcamp-sts-v1-train-5751,slack-rtt,"앞으로 추천팀과 전략팀이 어떻게 더 원활히 협업하고 서로에게 도움을 줄지 이야기 나누며,",추천팀과 전략팀이 앞으로 어떻게 더 잘 협력하고 서로를 도울 수 있는지에 대해 이야...,4.2,1.0,4.0
3,boostcamp-sts-v1-train-4196,slack-sampled,앞 : 으로 ! 잘 부탁드립니다!,앞으로 잘 부탁드립니다 . !!,4.8,1.0,5.0
4,boostcamp-sts-v1-train-4672,petition-sampled,강의보조인력을 늘려주십시오,대학입학에서 정시전형 비율을 늘려주십시오,0.0,0.0,0.0
...,...,...,...,...,...,...,...
17881,boostcamp-sts-v1-train-3583,nsmc-sampled,마지막 결말이 아쉬웟지만...,스토리도 좋았지만 결말이 안타깝네요.,2.8,1.0,3.0
17882,boostcamp-sts-v1-train-4274,nsmc-sampled,유일하게 내가 졸면서 보았던 영화.. 그것도 영화관에서.,영화관에서 아버지가 우는 모습을 처음 봤다.,1.0,0.0,1.0
17883,boostcamp-sts-v1-train-3311,petition-sampled,"이국종 교수 ? 의견 ? 을 , 적극 반영하세요",이국 ; 종 . 교수 ! 님을 적극 지원해 주세요,2.0,0.0,2.0
17884,boostcamp-sts-v1-train-6518,slack-rtt,"빨리 , 해봐야겠다고 생각 . 했어요!",조만간 먹어봐야겠다는 ; 생각을 했습니다 ; !ㅋㅋㅋ,1.0,0.0,1.0


다음 방법은 생성모델을 활용한 방법으로 Conditional BERT Contextual Augmentation 논문에 소개되었다. 

기존 BERT에서는 token embedding + segment embedding + positional embedding 으로 representation을 구성하지만, 

conditional BERT의 경우 token embedding + label embedding + positional embedding 으로 representation을 구성하고, 
label을 부착한 상태로 데이터셋을 MLM task로 pretraining한다. 
이후에 mask token을 replace하는 것과 마찬가지로 label에 대하여 token replace를 수행한다.